In [12]:
example = pd.read_csv(open('example.csv','rU'), encoding='utf-8', engine='c')
example.head()

,Unnamed: 0,Unnamed: 0.1,Apartment,Bathrooms,Bedrooms,Building_ID,Date_Created,Description,Address,latitude,...,Date_Created.Year,Date_Created.Date,Date_Created.WDayLbl,Date_Created.Week,Date_Created.Quarter,Date_Created.Hour,Date_Created.WeekofMonth,Date_Created.YDay,cleaned,Predicted_des
0,0,1,10.0,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:00,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,...,2016.0,2016-06-24,Fri,26.0,2.0,7.0,4.0,176.0,brand new bedroom bath apartmentenjoy apartmen...,medium
1,1,2,10000.0,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:00,,Columbus Avenue,40.7947,...,2016.0,2016-06-12,Sun,24.0,2.0,12.0,2.0,164.0,NaN,low
2,2,3,100004.0,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:00,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,...,2016.0,2016-04-17,Sun,16.0,2.0,3.0,3.0,108.0,top top west village location beautiful pre wa...,low
3,3,4,100007.0,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:00,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,...,2016.0,2016-04-18,Mon,16.0,2.0,2.0,3.0,109.0,building garden fitness room laundry room roof...,low
4,4,5,100013.0,1.0,4,NaN,2016-04-28 01:32:00,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,...,2016.0,2016-04-28,Thu,17.0,2.0,1.0,4.0,119.0,bedroom flex bedroom apartment rent available ...,low


In [70]:
trimDF = example[['Apartment','cleaned','Interest_Level']]
trimDF.head()

,Apartment,cleaned,Interest_Level
0,10.0,brand new bedroom bath apartmentenjoy apartmen...,medium
1,10000.0,NaN,low
2,100004.0,top top west village location beautiful pre wa...,high
3,100007.0,building garden fitness room laundry room roof...,low
4,100013.0,bedroom flex bedroom apartment rent available ...,low


In [71]:
trimDF.to_csv('trimDF.csv')

In [79]:
csv_file = raw_input('Enter the name of your input file: ')
txt_file = raw_input('Enter the name of your output file: ')

text_list = []

with open(csv_file, "r") as my_input_file:
    for line in my_input_file:
        line = line.split(",", 2)
        text_list.append(" ".join(line))

with open(txt_file, "w") as my_output_file:
    my_output_file.write("#1\n")
    my_output_file.write("double({},{})\n".format(len(text_list), 2))
    for line in text_list:
        my_output_file.write("  " + line)
    print('File Successfully written.')

Enter the name of your input file: trimDF.csv
Enter the name of your output file: trimDF.txt
File Successfully written.


In [86]:
from gensim.models import doc2vec
import nltk
import re

def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("trimDF.txt")):
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])

# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save('description.model')

In [87]:
model = doc2vec.Doc2Vec.load('description.model')

In [31]:
print model.most_similar(positive=["bedroom"], negative=["small"], topn=3)

[('ultra', 0.4796402156352997), ('penthouse', 0.4264823794364929), ('gorgeous', 0.40383094549179077)]


In [91]:
test = pd.read_csv('trimDF.csv')
test['docvecs'] = model.docvecs
test.head()

,Unnamed: 0,Apartment,cleaned,Interest_Level,docvecs
0,0,10.0,brand new bedroom bath apartmentenjoy apartmen...,medium,"[-0.0894624, 0.0398608, -0.0301156, 0.0530289,..."
1,1,10000.0,NaN,low,"[-0.00989012, 0.00795442, -0.00808452, 0.01144..."
2,2,100004.0,top top west village location beautiful pre wa...,high,"[-0.0491578, 0.0290913, 0.0248247, 0.0358007, ..."
3,3,100007.0,building garden fitness room laundry room roof...,low,"[-0.0338375, 0.0133059, 0.00771365, 0.0434318,..."
4,4,100013.0,bedroom flex bedroom apartment rent available ...,low,"[-0.0717796, 0.00993952, 0.0258335, 0.0218237,..."


In [89]:
len(test)

52800

In [90]:
len(model.docvecs)

52800

In [92]:
test['sum_vec'] = test['docvecs'].apply(lambda v: sum(v))
test.head()

,Unnamed: 0,Apartment,cleaned,Interest_Level,docvecs,sum_vec
0,0,10.0,brand new bedroom bath apartmentenjoy apartmen...,medium,"[-0.0894624, 0.0398608, -0.0301156, 0.0530289,...",-0.660609
1,1,10000.0,NaN,low,"[-0.00989012, 0.00795442, -0.00808452, 0.01144...",0.053875
2,2,100004.0,top top west village location beautiful pre wa...,high,"[-0.0491578, 0.0290913, 0.0248247, 0.0358007, ...",0.183141
3,3,100007.0,building garden fitness room laundry room roof...,low,"[-0.0338375, 0.0133059, 0.00771365, 0.0434318,...",-0.160693
4,4,100013.0,bedroom flex bedroom apartment rent available ...,low,"[-0.0717796, 0.00993952, 0.0258335, 0.0218237,...",-0.610769


In [101]:
description_vec = test[['Apartment','sum_vec']]
description_vec.head()

,Apartment,sum_vec
0,10.0,-0.660609
1,10000.0,0.053875
2,100004.0,0.183141
3,100007.0,-0.160693
4,100013.0,-0.610769


In [103]:
description_vec.to_csv('description_vec_v1.csv')